# COP501 CW

In [1]:
from database import *

from bikeSearch import *
from bikeRent import *
from bikeReturn import *
from bikeSelect import *

import ipywidgets as widgets

from datetime import date

In [2]:
database = Database('database9.db')

Connected to database9.db


In [3]:
def make_textbox(place_holder, descript):
    textbox = widgets.Text(
        value='',
        placeholder=place_holder,
        description=descript,
        disabled=False
    )
    return textbox

def clicked_search(btn):
    message = widgets.HTML(value='<h3>Search by brand, type or size for bikes!</h3>')

    search_term = widgets.Dropdown(options = ['Brand', 'Type', 'Size'],
                                description = 'Search by:')

    specific_search_term = widgets.Dropdown(options = ['Trek', 'Cannondale','Bianchi','Giant','Specialized'],
                                            description = 'Select')

    def update_specific_search_box(change):
        if search_term.value.lower() == 'brand':
            specific_search_term.options = ['Trek', 'Cannondale','Bianchi','Giant','Specialized']

        elif search_term.value.lower() == 'type':
            specific_search_term.options = ['Road Bike', 'Mountain Bike', 'Electric Bike']

        else:
            specific_search_term.options = ['Small', 'Medium', 'Large']

        
    search_term.observe(update_specific_search_box, names='value')

    submit_button = widgets.Button(description="Submit Search")
    
    # Function to execute when Submit button is clicked
    def on_submit(submit_btn):
        term_input = search_term.value.lower()
        param_input = specific_search_term.value.lower()
        # Call the BikeSearch function only after values are provided
        out = BikeSearch().search(database, term=term_input, parameter=param_input)
        with output:
            display(out)
        
    # Link the on_submit function to the Submit button
    submit_button.on_click(on_submit)
    with output:
        display(message, search_term, specific_search_term, submit_button)


def clicked_rent_return(btn):
    message = widgets.HTML(value='<h3>Rent your chosen bike:</h3>')
    member_id_textbox = make_textbox(place_holder = 'e.g., 1001',
                                     descript = 'Member ID:')
    bicycle_id_textbox = make_textbox(place_holder = 'e.g., 10',
                                      descript = 'Bicycle ID:')

    rent_date_textbox = make_textbox(place_holder = 'Format YYYY/MM/DD',
                                      descript = 'Rent date:')
    days_textbox = make_textbox(place_holder = 'e.g., 10',
                                      descript = 'No. of days?')

    submit_rent_button = widgets.Button(description="Rent")

    
    # Function to execute when Submit button is clicked
    def on_submit(submit_btn):
        memID_input = member_id_textbox.value
        bicID_input = bicycle_id_textbox.value

        rent_date = rent_date_textbox.value
        rental_time = days_textbox.value
        # Call the BikeSearch function only after values are provided
        out = BikeRent().rent(database, member_id = memID_input, bicycle_id = bicID_input,
                        rent_date_ = rent_date, rent_duration = rental_time)
        with output:    
            display(out)
        
    # Link the on_submit function to the Submit button
    submit_rent_button.on_click(on_submit)
    rent_vbox = widgets.VBox([message, member_id_textbox, bicycle_id_textbox, rent_date_textbox, days_textbox])

    ##############################################################################################

    message = widgets.HTML(value='<h3>Return your bike:</h3>')
    bicycle_id_textbox = make_textbox(place_holder = 'e.g., 10',
                                      descript = 'Bicycle ID:')

    return_date_textbox = make_textbox(place_holder = 'Format YYYY/MM/DD',
                                      descript = 'Return date')

    condition_dropdown = widgets.Dropdown(options = ['New', 'Good', 'Damaged'],
                                description = 'Condition')

    submit_button = widgets.Button(description="Return")

    # Function to execute when Submit button is clicked
    def on_submit(submit_btn):
        bicID_input = bicycle_id_textbox.value
        actual_return = return_date_textbox.value
        cond_input = condition_dropdown.value.lower()
        
        # Call the BikeSearch function only after values are provided
        out = BikeReturn().return_bike(database, bicycle_id = bicID_input, 
                                actual_return_date = actual_return, 
                                condition = cond_input)
        with output:
            display(out)
        
    # Link the on_submit function to the Submit button
    submit_button.on_click(on_submit)
    

    return_vbox = widgets.VBox([message, bicycle_id_textbox, return_date_textbox, condition_dropdown])
    submit_hbox = widgets.HBox([submit_rent_button, submit_button])
    overall_output = widgets.HBox([rent_vbox, return_vbox])
    
    with output:
        display(overall_output, submit_hbox)


def clicked_select(btn):
    #could do a dropdown box here?
    title = widgets.HTML(value='<h3>Select bikes for stock maintenance or expansion:</h3>')
    message = widgets.HTML(value='<p>Choose what percentage you wish to allocate your budget to maintaing your inventory.<p> The rest of your budget then goes to expanding your inventory with new bikes.</p>')
    budget_allocation = widgets.IntSlider(value=100, min=0, max=100, step=1,
                                description='Percent:',
                                disabled=False,
                                continuous_update=False,
                                orientation='horizontal',
                                readout=True, readout_format='d')
    
    overall_budget = widgets.IntSlider(value=20000, min=0, max=20000, step=100,
                                description='Budget £',
                                disabled=False,
                                continuous_update=False,
                                orientation='horizontal',
                                readout=True, readout_format='d')

    submit_button = widgets.Button(description="Submit Search")
    
    # Function to execute when Submit button is clicked
    def on_submit(submit_btn):
        budget_input = overall_budget.value
        allocation_input = budget_allocation.value
        # Call the BikeSearch function only after values are provided
        out = BikeSelect(database).select(database, allocation=allocation_input, budget=budget_input)
        with output:
            display(out[0], out[1])
        
    # Link the on_submit function to the Submit button
    submit_button.on_click(on_submit)
    with output:
        display(title, message, overall_budget, budget_allocation, submit_button)


def clicked_visualise(btn):
    out = BikeSelect(database).visualise()

   
output=widgets.Output()

display(widgets.HTML(value="<h1>Welcome to the Bicycle Rental Management System!</h1><h4>Please select an option below</h4>"))

buttons = {'Search': clicked_search , 'Rent & Return' : clicked_rent_return, 'Select' : clicked_select, 'Visualise':clicked_visualise}
widgets_list=[]
for btn, function in buttons.items():
    btn = widgets.Button(description = btn)
    btn.on_click(function)
    widgets_list.append(btn)


button_box = widgets.HBox(widgets_list)
widgets.VBox([button_box, output])

HTML(value='<h1>Welcome to the Bicycle Rental Management System!</h1><h4>Please select an option below</h4>')

In [4]:
# database = Database('database.db')
# database.clear_db()

In [5]:
# tables={'rental_hist': 'Rental_History.txt', 'bicycles': 'Bicycle_info.txt'}
# for name,path in tables.items():
#     database.create_table(name)
#     database.clean_load_files_to_table(name, path)

In [6]:
#BikeSearch().search(database, term = 'type', parameter = 'Mountain Bike')